### (1) read training file: extract (image, label) tuples

In [ ]:
with open('../../data/isbi2016_isic_part1_train.list', 'r') as training_file:
    images_labels_train = training_file.readlines()
    
images_train=[]
labels_train=[]
for image_label in images_labels_train:
    i_l = image_label.split()
    if "_01" not in i_l[0]:
        images_train.append(i_l[0])
        labels_train.append(i_l[1])
    
print(dict(zip(images_train,labels_train)))

### (2) read validation file: extract (image, label) tuples

In [ ]:
with open('../../data/isbi2016_isic_part1_val.list', 'r') as validation_file:
    images_labels_val = validation_file.readlines()

images_val=[]
labels_val=[]
for image_label in images_labels_val:
    i_l = image_label.split()
    if "_01" not in i_l[0]:
        images_val.append(i_l[0])
        labels_val.append(i_l[1])
    
print(dict(zip(images_val,labels_val)))

### (3) compute per-channel mean and build tfrecords from training images

In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
import os
import io
import PIL import ImageStat

append_dir = '../../data/ISBI2016_ISIC_Part1_Training_Data_processed/'

mean_rgb = np.array([0., 0., 0.])
count=0

def _bytes_feature(value): 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

writer = tf.python_io.TFRecordWriter('melanoma_train.tfrecords')

for fimg, flabel in zip(images_train,labels_train):
    print(fimg, flabel)
    bytes_io_img = io.BytesIO()
    img = Image.open(os.path.join(append_dir,fimg))
    stat = ImageStat.Stat(img)
    mean_rgb = mean_rgb + np.array(stat.mean)
    count += 1
    img.save(bytes_io_img, 'PNG')
    bytes_io_lbl = io.BytesIO()
    label = Image.open(os.path.join(append_dir,flabel))
    label.save(bytes_io_lbl, 'PNG')
    example = tf.train.Example(
        features=tf.train.Features(
            feature={'image': _bytes_feature(bytes_io_img.getvalue()),
                     'image_name': _bytes_feature(str.encode(fimg)),
                     'label_name': _bytes_feature(str.encode(flabel)),
                     'label': _bytes_feature(bytes_io_lbl.getvalue())
                    }
        )
    )
    writer.write(example.SerializeToString())
writer.close()

np.save('mean_rgb_train.npy', mean_rgb/count)

SyntaxError: invalid syntax (<ipython-input-1-89d1c13365ea>, line 6)

In [ ]:
writer = tf.python_io.TFRecordWriter('melanoma_val.tfrecords')

for fimg, flabel in zip(images_val,labels_val):
    print(fimg, flabel)
    bytes_io_img = io.BytesIO()
    img = Image.open(os.path.join(append_dir,fimg))
    img.save(bytes_io_img, 'PNG')
    bytes_io_lbl = io.BytesIO()
    label = Image.open(os.path.join(append_dir,flabel))
    label.save(bytes_io_lbl, 'PNG')
    example = tf.train.Example(
        features=tf.train.Features(
            feature={'image': _bytes_feature(bytes_io_img.getvalue()),
                     'image_name': _bytes_feature(str.encode(fimg)),
                     'label_name': _bytes_feature(str.encode(flabel)),
                     'label': _bytes_feature(bytes_io_lbl.getvalue())
                    }
        )
    )
    writer.write(example.SerializeToString())
writer.close()